In [1]:
from sympy.physics.quantum import Commutator, Dagger, Operator
import sympy
from sympy import *
from IPython.display import display
import numpy as np

# Hamiltonian

Define operators and constants

In [2]:
U, F, delta_a, delta_b, J = symbols(r'U F \Delta_a \Delta_b J')

a = Symbol(r'a', commutative=False)
ad = Symbol(r'a^\dag', commutative=False)
b = Symbol(r'b', commutative=False)
bd = Symbol(r'b^\dag', commutative=False)

rho = Symbol(r'\rho', commutative=False)

In [3]:
a.args 
dd = a.args
dd2 = (a*ad).args

print(dd)
print(dd2)

print(len(dd))
print(len(dd2))

()
(a, a^\dag)
0
2


Construct Hamiltonian

In [4]:
Ham1 = J*(ad*b+a*bd) + F*(ad+a) -delta_a*ad*a -delta_b*bd*b #Rational(1, 2)*U*ad*ad*a*a
Ham2 = F*(ad+a*b)
Ham3 = Mul(U,ad,ad,a,a,evaluate=False)
Ham4 = UnevaluatedExpr(Mul(Rational(1, 2)*U,ad,ad,a,a,evaluate=False))


MyHam = Ham2
display(MyHam)

F*(a*b + a^\dag)

Jump operators

# Lindblad master equation

In [5]:
ME_comm = -sympy.I* (Commutator(MyHam, rho))
display(ME_comm)
ME= ME_comm.doit()
#display(ME)
ME = expand(ME)
display(ME)

I*F*[\rho,a*b + a^\dag]

I*F*\rho*a*b + I*F*\rho*a^\dag - I*F*a*b*\rho - I*F*a^\dag*\rho

In [6]:
#ME = ad*a*rho+rho*ad*a

# Convert to desired phase space function

In [7]:
from phasespaceconversion import pow_to_mul_sep2,PhaseSpaceFunction

Wigner

In [8]:
myWigner = PhaseSpaceFunction(ME,'W')
W_eqn = myWigner.getFPfromME()
print('Result')
display(expand(W_eqn))

Term 0 :


I*F*\rho*a^\dag

constants in term:


I*F

\rho*a^\dag

...

a*b

[a, b]
LHS term, order operators applied:
[b, a]


-I*F*(\alpha*(\beta*W(\alpha, \alpha^*, \beta, \beta^*) + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)/2) + \beta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)/2 + Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*, \beta^*)/4)

Result


-I*F*\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*) - I*F*\beta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)

HusimiQ

In [9]:
myQ= PhaseSpaceFunction(ME,'Q')
Q_eqn = myQ.getFPfromME()
print('Result')
display(expand(Q_eqn))

Term 0 :


I*F*\rho*a^\dag

constants in term:


I*F

\rho*a^\dag

...

a*b

[a, b]
LHS term, order operators applied:
[b, a]


-I*F*(\alpha*(\beta*Q(\alpha, \alpha^*, \beta, \beta^*) + Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \beta^*)) + \beta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*, \beta^*))

Result


-I*F*\alpha*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \beta^*) - I*F*\beta*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) - I*F*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha) - I*F*Derivative(Q(\alpha, \alpha^*, \beta, \beta^*), \alpha^*, \beta^*)

P representation

In [10]:
myP= PhaseSpaceFunction(ME,'P')
P_eqn = myP.getFPfromME()
print('Result')
display(expand(P_eqn))

Term 0 :


I*F*\rho*a^\dag

constants in term:


I*F

\rho*a^\dag

...

a*b

[a, b]
LHS term, order operators applied:
[b, a]


-I*F*\alpha*\beta*P(\alpha, \alpha^*, \beta, \beta^*)

Result


-I*F*\alpha*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \beta^*) - I*F*\beta*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*) + I*F*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha) + I*F*Derivative(P(\alpha, \alpha^*, \beta, \beta^*), \alpha^*, \beta^*)

# Separate Real and imaginary Parts Wigner

## Not working for now, use mathematica file instead

In [11]:
W_exp_comp = expand(W_eqn)
display(W_exp_comp)

# redefine symbols in original equation (not so nice that this has to be repeated but okay)

#field = Function('W')(alpha,self.alphas,self.beta,self.betas) # can be W, Q or P
alpha = Symbol(r'\alpha', commutative=True)
alphas = Symbol(r'\alpha^*', commutative=True)
beta = Symbol(r'\beta', commutative=True)
betas = Symbol(r'\beta^*', commutative=True)

# define real and imag parts:
alpha_r = Symbol(r'a_R', commutative=True)
alpha_i = Symbol(r'a_I', commutative=True)
beta_r = Symbol(r'b_R', commutative=True)
beta_i = Symbol(r'b_I', commutative=True)

# substitute in expression
W_exp_comp_sep =W_exp_comp.subs(alpha, alpha_r+sympy.I*alpha_i )
W_exp_comp_sep =W_exp_comp.subs(alphas, alpha_r-sympy.I*alpha_i )
W_exp_comp_sep = W_exp_comp.subs(beta, beta_r+sympy.I*beta_i )
W_exp_comp_sep = W_exp_comp.subs(betas, beta_r-sympy.I*beta_i )

display(W_exp_comp_sep)
W_exp_Re = re(W_exp_comp)
display(W_exp_Re)

-I*F*\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*) - I*F*\beta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*)

-I*F*\alpha*Subs(Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*), \beta^*, -I*b_I + b_R) - I*F*\beta*Derivative(W(\alpha, \alpha^*, \beta, -I*b_I + b_R), \alpha^*)

im(F*\alpha*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \beta^*)) + im(F*\beta*Derivative(W(\alpha, \alpha^*, \beta, \beta^*), \alpha^*))

In [12]:
W_exp_comp = expand(W_eqn)
W_exp_comp.atoms()
W_exp_comp.atoms(Function)

{W(\alpha, \alpha^*, \beta, \beta^*)}